# RBAC Interview Bible 
## Python for Data Science

In [ ]:
# loading dataframe
df = pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [3]:
df.head()

****EDA****

In [4]:
df.info()

In [5]:
## cleaning missing values
df.shape

In [6]:
df[df.duplicated()] ## no duplicates

In [7]:
#df['ever_married'] = df['ever_married'].map({'Yes': 1, 'No': 0})

In [8]:
df['gender'].value_counts()

In [9]:
#df['gender'] = df['gender'].replace({'Female': 1, 'Male': 0, 'Other': 2})
# 2 - Other
# 1 - Female
# 0 - Male

In [10]:
df.head()

In [11]:
df.info()

In [12]:
df.work_type.value_counts()

In [13]:
df.Residence_type.value_counts()
# almost equal number of people from rural and urban areas in the 

In [14]:
df['avg_glucose_level'].describe()

In [15]:
df['bmi'].describe()

In [16]:
df['smoking_status'].value_counts()

In [17]:
df['stroke'].value_counts()

In [18]:
## handling missing data - BMI - by Prediction through decision tree - variables used: age and gneder


from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder



DT_bmi_pipeline = Pipeline(steps = [
    ('scale', StandardScaler()),
    ('lr', DecisionTreeRegressor(random_state = 42))
])


X = df[['age', 'gender', 'bmi']].copy()
X.gender = X.gender.replace({'Male': 0, 'Female': 1, 'Other': -1})

X.gender = X.gender.astype(np.uint8)


In [19]:
## Separting those values that have Missing BMI

# test set would contain all bmi missing 201 values
Missing = X[X.bmi.isna()]

# train set would contain all non-missing bmi values
X_train = X[~X.bmi.isna()]
Y_train = X_train.pop('bmi')

# Fit regressor
DT_bmi_pipeline.fit(X_train, Y_train)

In [20]:
## Predicting missing BMI and substituting in the original dataset
predicted_bmi = pd.Series(DT_bmi_pipeline.predict(Missing[['age', 'gender']]), index = Missing.index)
predicted_bmi

In [21]:
# substituting in original dataset
df.loc[Missing.index, 'bmi'] = predicted_bmi

In [22]:
df.isnull().sum()

## PLOTS TO UNDERSTAND CORRELATIONS

In [23]:
df.corr(method = 'pearson')

In [24]:
variables = [variable for variable in df.columns]

conts = ['age','avg_glucose_level','bmi']

In [25]:
variables

In [26]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 12), facecolor='#f6f6f6')
gs = fig.add_gridspec(4, 3)
gs.update(wspace=0.1, hspace=0.4)
gs

In [27]:
corr = df.corr()

In [28]:
mask = np.triu(np.ones_like(corr, dtype=bool))
mask

In [29]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

import seaborn as sns
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

## Age, hypertension, heartdisease, ever_married, avg_glucose_level are the most correlated

In [30]:
# singular plots


# hypertension
# heart_disease
# ever_married
# gender
# stroke
# smoking


sns.set_theme(style = 'darkgrid')
#plot_hypertension = sns.countplot(data = df, x = 'hypertension')
#plot_heart_disease = sns.countplot(data = df, x = 'heart_disease')
plot = sns.countplot(data = df, x = 'work_type')

In [31]:
# For Avergae Glucose level
fig = plt.figure(figsize=(7,7))
sns.displot(df.avg_glucose_level, color="green", label="avg_glucose_level", kde= True)
plt.legend()

In [32]:
# For BMI distribution
fig = plt.figure(figsize=(10,10))
sns.displot(df.bmi, color = 'blue', label = 'BMI', kde = True)
plt.legend()

In [33]:
## Stroke vs No Stroke Avergae Glucose Level
plt.figure(figsize=(12,10))

sns.distplot(df[df['stroke'] == 0]["bmi"], color='green') # No Stroke - green
sns.distplot(df[df['stroke'] == 1]["bmi"], color='red') # Stroke - Red

plt.title('Stroke vs No Stroke BMI', fontsize = 15)
plt.xlim([10,100])
plt.show()

In [34]:
plt.figure(figsize=(12,10))
sns.distplot(df[df['stroke']==0]['avg_glucose_level'], color = 'green')
sns.distplot(df[df['stroke'] == 1]['avg_glucose_level'], color = 'red')
plt.title('Stroke vs No stroke Average Glucose Level', fontsize = 15)
plt.xlim([30,300]) ## limit of x-axis
plt.show()

# Scatter Plot to see 3 variable variation

In [35]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data = df, x = "age", y = "bmi", hue = "gender")
## to draw a line within the graph
graph.axhline(y = 25, linewidth = 4, color = "r", linestyle = '--')
graph.axhline()
plt.show()

In [36]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data= df, x = "age", y = "avg_glucose_level", hue="gender")
graph.axhline(y = 150, linewidth = 4, color = "r", linestyle = "--")
fig.show()

According to the graph, all people above glucose level 150 and above BMI 25 are considered as having more risk to Stroke

In [37]:
# With respect to smokers and non-smokers
df.head()

In [38]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data = df, x = "smoking_status", y = "avg_glucose_level")
fig.show()

In [39]:
# pair plot - all possible scatter plots
fig = plt.figure(figsize=(10,10))
sns.pairplot(df)
plt.show()

# Data Preprocessing

In [40]:
# Getting predictor and the target variables separate
x = df.iloc[:, 1:-1].values
x

In [41]:
y = df.iloc[:, -1].values
y

In [42]:
## One hot encoding for categorical variables: gender, work_type, smoking_status
## Label encoding of binary variables: ever_married, residence_type

In [43]:
df.rename(columns= {'Residence_type': 'residence_type'}, inplace=True)

In [44]:
df.head()

In [45]:
df['residence_type'].value_counts()

In [46]:
# one hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

ct = ColumnTransformer(transformers= [('encoder', ohe, [0])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))
x

In [47]:
x = np.delete(x, 2, 1) # deleting the 3rd column so as to delete one gender one hot encoded variable to prevent multicollinearity

In [48]:
x[0][6]

In [49]:
# one hot encoding of work_type
ct = ColumnTransformer(transformers = [('encoder', ohe, [6])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))
x

In [50]:
print("This should be private: ", x[0][2])
print("This should also be private: ", x[2][2])
print("This should also be private: ", x[3][2])

In [51]:
# Where does the work_type end
# 0,1: gender
# 2,3,4,5,6: work_type
x[0][7] # age

In [52]:
# drop number 6 of work_type
x = np.delete(x, 6, 1)
x[0][6]

In [53]:
# what is the current index of smoking status
x[0][13]

In [54]:
# One hot encoding of smoking status
ct = ColumnTransformer(transformers = [('encoder', ohe, [13])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))
x

In [55]:
# 0,1 : gender
# 2,3,4,5: work_type
# 6,7,8,9: smoking status
x = np.delete(x, 9, 1)
x.shape

# Final Tally:

### 0,1 index are gender
### 2,3,4,5 are work_type
### 6,7,8: smoking status

### Ellimination of one hot encoded variable per predictor is done to avoid Multicollineartity

In [56]:
# Label Encoding
i,j = np.where(x == "Urban")

In [57]:
i # row number

In [58]:
j # Column number

In [59]:
#Index 15 is ever_married and index 16 is residence_type
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 12] = le.fit_transform(x[:, 12])
x[:, 13] = le.fit_transform(x[:, 13])
x

In [60]:
print("Shape x: ", x.shape) 
print("Shape y: ", y.shape)

In [61]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [62]:
print("Number of records in training set: ", x_train.shape)
print("Shape of training set's labels: ", y_train.shape)
print("Number of records in test set: ", x_test.shape)
print("Shape of test set's labels: ", y_test.shape)

## Feature Scaling

##### StandardScaler does normalization (x-x_bar)/std_dev --> converts variables into normal dist. with mean 0 and standard dev = 1

In [63]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [64]:
x_test

## Handling Imbalanced Data using SMOTE

In [65]:
print("Shape of training data: {} and {}".format(x_train.shape, y_train.shape))

### SMOTE is Synthetic Minority Oversampling Technique - overcoming the overfitting problem and class imbalance problem by generating synthetic minority records by random over-sampling

In [66]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 0)
x_train_res, y_train_res = sm.fit_resample(x_train, y_train.ravel())

In [67]:
x_train_res.shape

In [68]:
print("Shape of training data after synthetic oversampling: {} and {} ".format(x_train_res.shape, y_train_res.shape))

In [69]:
print("Number of label 1 records before over_sampling: ", sum(y_train == 1))
print("Number of label 0 records before over_sampling: ", sum(y_train == 0))
print("\n")
print("Number of label 1 records before SMOTE: ", sum(y_train_res == 1))
print("Number of label 0 records after SMOTE: ", sum(y_train_res == 0))

In [70]:
x_train_res

# Testing out Baseline Classification Models before Tuning

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [72]:
# importing all metrics
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay,precision_score, recall_score, f1_score, classification_report, roc_auc_score, roc_curve, plot_roc_curve, precision_recall_curve, plot_precision_recall_curve, auc, average_precision_score

from sklearn.model_selection import cross_val_score

In [73]:
models_array = []

## Adding it in an array so that could be made into a dataframe later
models_array.append(['Logistic Regression', LogisticRegression(random_state = 0)])
models_array.append(['K Nearest Neighbours', KNeighborsClassifier()])
models_array.append(['SVM Classifier', SVC(random_state=0)])
models_array.append(['Gaussian Naive Bayes', GaussianNB()])
models_array.append(['Bernoulli Naive Bayes', BernoulliNB()])
models_array.append(['Decision Tree Classifier', DecisionTreeClassifier(random_state = 0)])
models_array.append(['Random Forest', RandomForestClassifier(random_state = 0)])
models_array.append(['XGBoost Classifier', XGBClassifier(eval_metric = 'error',use_label_encoder=False)])

In [74]:
list1= []

for m in range(len(models_array)):
    list2 = []
    model = models_array[m][1]
    model.fit(x_train_res, y_train_res)
    y_pred = model.predict(x_test)
    
    # confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # training accuracies - K-fold cross validation
    accuracies = cross_val_score(estimator = model, X = x_train_res, y = y_train_res, cv = 10)
    
    # Accuracy score of test set
    accuracy = accuracy_score(y_test, y_pred)
    
    # Area Score under Reciever Operating Characteristic Curve
    roc = roc_auc_score(y_test, y_pred) 
    
    # Precision
    precision = precision_score(y_test, y_pred)
    
    # Recall
    recall = recall_score(y_test, y_pred)
    
    # F1-score
    f1 = f1_score(y_test, y_pred)
    
    # Printing the results for every model
    print(models_array[m][0], ":")
    print("Training set mean accuracy: {:.2f} %".format(accuracies.mean()*100))
    print("Training set standard deviation: {:.2f} %".format(accuracies.std()*100))
    print("Test set accuracy: {:.2f} %".format(accuracy*100))
    print("Confusion Matrix: ", cm)
    print("Precision: {:.2f} %".format(precision*100))
    print("Recall: {:.2f} %".format(recall*100))
    print("F1 score: {:.2f}".format(f1))
    print("AUROC score: {:.2f}".format(roc))
    print("-----------------------------------------")
    print("\n")
    
    
    ## Adding all scores to a list so that we can form a dataframe later
    list2.append(models_array[m][0])
    list2.append(accuracies.mean()*100)
    list2.append(accuracies.std()*100)
    list2.append(accuracy)
    list2.append(precision)
    list2.append(recall)
    list2.append(f1)
    list2.append(roc)
    
    list1.append(list2) ## now list1 contains all the information of the results of the models - this can be put in a data frame
    
    

From above results we can see that on training and test set the following models performed well:
1. KNN
2. Decision Tree
3. Random Forest
4. XGBoost


The following performed poorly:
1. SVM
2. Logistics Regression
3. Naive Bayes (both Gausian and Bernoulli)

However, before eliminating any models, we should try to select the best hyperparameter and see the performance of each

In [75]:
## Adding the results in a dataframe
df_results = pd.DataFrame(list1, columns = ['Model','Training Mean Accuracy', 'Training Standard Deviation', 'Test Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC Value'])
df_results.sort_values(by = ['Test Accuracy', 'Training Mean Accuracy', 'AUC Value'], inplace=True, ascending = False)
df_results

# Tuning Models

In [76]:
# Grid search to select hyper-parameters 
from sklearn.model_selection import GridSearchCV

#### Hyperparameter C is used to prevent overfitting and provide regularization strength:
#### For more depth, I refered to this stackoverflow link: https://stackoverflow.com/questions/22851316/what-is-the-inverse-of-regularization-strength-in-logistic-regression-how-shoul


#### var_smoothing in Gaussian Naive Bayes artificially adds a user-defined standard deviation on how many samples to be included near or away from the mean in a gaussian distribution. Link: https://stackoverflow.com/questions/58046129/can-someone-give-a-good-math-stats-explanation-as-to-what-the-parameter-var-smoo


#### cv in grid search means the number of cross validation we have for each set of hyperparameters


In [77]:
grid_models = [
    (LogisticRegression(), [{'C':[0.25, 0.5, 0.75, 1], 'random_state':[0]}]),
    #(KNeighborsClassifier(), [{'n_neighbors': [5,7,8,10], 'metric': ['euclidian', 'manhattan', 'chebyshev', 'minkowski']}]),
    (SVC(), [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear', 'rbf'], 'random_state':[0]}]),
    (GaussianNB(), [{'var_smoothing': [1e-09]}]),
    (BernoulliNB(), [{'alpha': [0.25, 0.5, 1]}]),
    (DecisionTreeClassifier(), [{'criterion': ['gini', 'entropy'], 'random_state': [0]}]),
    (RandomForestClassifier(), [{'n_estimators': [100, 150, 200], 'criterion': ['gini', 'entropy'], 'random_state': [0]}]),
    (XGBClassifier(), [{'learning_rate': [0.01, 0.05, 0.1], 'eval_metric': ['error'], 'use_label_encoder': [False]}])
]

In [78]:
for i,j in grid_models:
    grid = GridSearchCV(estimator = i, param_grid = j, scoring = 'accuracy', cv = 10)
    grid.fit(x_train_res, y_train_res)
    best_accuracy = grid.best_score_
    best_param = grid.best_params_
    
    print("{}: ", i)
    print("Best Accuracy:  {:.2f} %".format(best_accuracy*100))
    print("Best parameters selected: ", best_param)
    print("------------------------------")
    print("\n")
    

# Random Forest and XGBoost after hyperparmarater tuning

## Random Forest

In [79]:
# Fitting the random forest model with the above hyper parameters obtained by grid search

classifier = RandomForestClassifier(criterion = 'entropy', n_estimators = 150, random_state = 0)
classifier.fit(x_train_res, y_train_res)
y_pred = classifier.predict(x_test)
y_prob = classifier.predict_proba(x_test)[:,1] #probability with which the prediction is made
cm = confusion_matrix(y_test, y_pred)

print("predicted y: ", y_pred)

In [80]:
cm

In [81]:
rf_results_df = pd.DataFrame({'Predicted Stroke': y_pred, 
                             'Predicted Stroke Probability': y_prob,
                             'Actual Stroke': y_test})
rf_results_df.head()

In [82]:
print(classification_report(y_test, y_pred))
print("ROC AUC Score: ", roc_auc_score(y_test, y_prob))
print("Test Accuracy: ", accuracy_score(y_test, y_pred))

In [83]:
# Visualizing Confusion Matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,10))
sns.heatmap(cm, cmap = 'Blues', annot = True, fmt='d', linewidths = 5, cbar = False,
           annot_kws = {'fontsize': 15}, yticklabels= ['No Stroke', 'Stroke'], 
           xticklabels= ['Predicted No Stroke', 'Predicted Stroke'])
plt.yticks(rotation = 0)
plt.show()

In [84]:
# Visualizing the ROC-AUC curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc_score = auc(false_positive_rate, true_positive_rate)
roc1 = roc_auc_score
print("AUC of tuned RF is: ", roc_auc_score)

In [85]:
# Visualizing the ROC curve

sns.set_theme(style = 'white')
plt.figure(figsize = (10,10))
plt.plot(false_positive_rate, true_positive_rate, color='#b01717', label = 'AUC = %0.4f' %roc_auc_score)
plt.legend(loc = 'lower right')
plt.plot([0,1], [0,1], linestyle='--', color = '#174ab0')
plt.axis('tight') # to remove additional white space in the plot
plt.ylabel('True Positive Rate (Sensitivity)')
plt.xlabel('False Positive rate (1-Specificity)')
plt.legend()
plt.show()

## XG Boost with tuning

In [86]:
classifier = XGBClassifier(eval_metric = 'error', use_label_encoder = False, learning_rate = 0.1)
classifier.fit(x_train_res, y_train_res)
y_pred = classifier.predict(x_test)
y_prob = classifier.predict_proba(x_test)[:,1]
cm = confusion_matrix(y_test, y_pred)

In [87]:
# Putting the results in a dataframe
df_xgboost_results = pd.DataFrame({'Predicted Stroke Variable': y_pred, 
                                  'Predicted Stroke Probability': y_prob,
                                  'Actual Stroke Variable': y_test})
df_xgboost_results.head()

In [88]:
cm

In [89]:
accuracy_score(y_test, y_pred)

In [90]:
# ROC AUC curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc_score = auc(false_positive_rate, true_positive_rate)
roc_auc_score

In [91]:
## Visualization - Confusion Matrix
plt.figure(figsize = (10,10))
sns.heatmap(cm, cmap = "Blues", annot = True, fmt = 'd', linewidth = 5, cbar = False, annot_kws = {'fontsize': 15},
           xticklabels = ['Predicted No Stroke', 'Predicted Stroke'], yticklabels = ['No Stroke', 'Stroke'])
plt.show()

In [92]:
## Visualizaton - ROC AUC Curve

sns.set_theme(style = 'white')
plt.figure(figsize = (10,10))
plt.plot(false_positive_rate, true_positive_rate, color='#b01717', label = 'AUC = %0.4f' %roc_auc_score)
plt.legend(loc = 'lower right')
plt.plot([0,1], [0,1], linestyle='--', color = '#174ab0')
plt.axis('tight') # to remove additional white space in the plot
plt.ylabel('True Positive Rate (Sensitivity)')
plt.xlabel('False Positive rate (1-Specificity)')
plt.legend()
plt.show()

# Summary of Results

Clearly the 3 good performing models for Stroke Prediction are as follows:
1. Random Forest: accuracy: 0.89 , auc: 0.75
2. XG Boost: accuracy: 0.87 , auc: 0.74

So a tuned Random Forest performed best for this data